In [ ]:
from keras.preprocessing.image import ImageDataGenerator


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,shear_range = 0.2,zoom_range = 0.2)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/data/train_set',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')


Found 316 images belonging to 3 classes.


In [ ]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/data/test_set',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 120 images belonging to 3 classes.


In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [ ]:
IMAGE_SIZE = [224,224]

train_path = '/content/drive/MyDrive/data/train_set'
valid_path = '/content/drive/MyDrive/data/test_set'

In [ ]:
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)


58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
for layer in vgg16.layers:
  layer.trainable = False

In [ ]:
folders = glob('/content/drive/MyDrive/data/train_set/*')

In [ ]:
folders

['/content/drive/MyDrive/data/train_set/crack',
 '/content/drive/MyDrive/data/train_set/flakes',
 '/content/drive/MyDrive/data/train_set/roof']

In [ ]:
x = Flatten()(vgg16.output)


In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)

In [ ]:
model = Model(inputs=vgg16.input, outputs=prediction)


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
) 

In [ ]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

<ipython-input-52-5932e3024fde>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/10
10/10 [==============================] - 396s 40s/step - loss: 1.3642 - accuracy: 0.4715 - val_loss: 1.1989 - val_accuracy: 0.5583
Epoch 2/10
10/10 [==============================] - 314s 32s/step - loss: 0.5869 - accuracy: 0.7785 - val_loss: 0.8885 - val_accuracy: 0.6667
Epoch 3/10
10/10 [==============================] - 368s 38s/step - loss: 0.3955 - accuracy: 0.8481 - val_loss: 0.7540 - val_accuracy: 0.7167
Epoch 4/10
10/10 [==============================] - 372s 39s/step - loss: 0.2495 - accuracy: 0.9114 - val_loss: 0.6256 - val_accuracy: 0.7500
Epoch 5/10
10/10 [==============================] - 367s 38s/step - loss: 0.1919 - accuracy: 0.9399 - val_loss: 0.6182 - val_accuracy: 0.7833
Epoch 6/10
10/10 [==============================] - 372s 39s/step - loss: 0.1397 - accuracy: 0.9589 - val_loss: 0.5855 - val_accuracy: 0.7917
Epoch 7/10
10/10 [==============================] - 309s 32s/step - loss: 0.1139 - accuracy: 0.9810 - val_loss: 0.6036 - val_accuracy: 0.7667
Epoch 

In [ ]:
from tensorflow.keras.models import load_model
model.save('model_building_defects_vgg16.h5')

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
model = load_model("model_building_defects_vgg16.h5")

In [191]:
img=image.load_img(r'/content/drive/MyDrive/data/test_set/flakes/8.jpg')
img = img.resize((224, 224))
x=image.img_to_array(img)

In [192]:
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)

In [193]:
a=np.argmax(model.predict(img_data), axis=1)

1/1 [==============================] - 1s 808ms/step


In [194]:
index=['crack','flakes','roof']
result=str(index[a[0]])
result    

'flakes'